In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge




import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer


In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


<Experiment: artifact_location='file:///d:/MLOPS/mlruns/1', creation_time=1697124807956, experiment_id='1', last_update_time=1697124807956, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(url_):
    data = pd.read_csv(url_)
    data['lpep_pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'])
    data['lpep_dropoff_datetime'] = pd.to_datetime(data['lpep_dropoff_datetime'])

    data['duration'] = data['lpep_dropoff_datetime'] - data['lpep_pickup_datetime']

    data['duration'] = data['duration'].apply(lambda td : td.total_seconds()/60)

    categorical = ['PULocationID', 'DOLocationID']
    data[categorical] = data[categorical].astype(str)

    return data

In [6]:
#url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"


#url1 = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"


data_train =  read_dataframe('D:\MLOPS\january_data.csv')
data_val = read_dataframe('D:\MLOPS\efebruaury_data.csv')

C:\Users\mohdf\AppData\Local\Temp\ipykernel_17588\869422538.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url_)


In [7]:
len(data_train) , len(data_val)

(76518, 64572)

In [8]:
data_train["PU_DO"] = data_train['PULocationID'].astype(str) + '_' + data_train['DOLocationID'].astype(str)
data_val['PU_DO'] = data_val['PULocationID'].astype(str) + '_' + data_val['DOLocationID'].astype(str)

In [9]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dict  = data_train[categorical + numerical].to_dict(orient = 'records')

X_train = dv.fit_transform(train_dict)

val_dicts = data_val[categorical + numerical].to_dict(orient = 'records')

X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = data_train[target].values
y_val = data_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

53.60353015809937

In [12]:
ls = Lasso(alpha = 0.001)
ls.fit(X_train, y_train)

y_pred = ls.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

54.536784814143324

In [13]:
rg = Ridge(alpha = 0.1)
rg.fit(X_train, y_train)

y_pred = rg.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

54.28622822691404

In [14]:
with open('D:\MLOPS\lin_reg.bin','wb') as f_out:
  pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    mlflow.set_tag("developer","faaiz")

    mlflow.log_param("train-data-path","D:\MLOPS\january_data.csv")
    mlflow.log_param("val-data-path",'D:\MLOPS\efebruaury_data.csv')

    alpha = 0.01

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared= False)

    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="D:\MLOPS\lin_reg.bin",artifact_path="model_pickle")

    


In [16]:
import xgboost as xgb 
from hyperopt import fmin, tpe , hp , STATUS_OK , Trials
"""
fmin = will try to minimize the output.
tpe = algorithm used to control the logic,
hp = contains difffrent method that define search space.
STATUS-OK = signal we will sent at the end of each run.
 
"""
from hyperopt.pyll import scope



In [17]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)



In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)  #parameters will be locked to mlflow

        booster = xgb.train(
            params = params, #passing parameters in training of a model
            dtrain = train, # passing traning data
            num_boost_round = 1000, # setting maximum 1000 etirations.
            evals = [(valid , 'validation')], #using validation set to control the optimization algorithm from xgboost.
            early_stopping_rounds =  50 # if there are 50 or more iteration without improvment on validation error, then the optimization of xgboost will be stop
        )

        y_pred = booster.predict(valid)  # mzking prediction on validation set.

        rmse = mean_squared_error(y_val, y_pred, squared = False) # eror on validation set

        mlflow.log_metric("rmse",rmse)

    return({'loss': rmse, 'status': STATUS_OK}) #returning loss and status
    

In [19]:
search_space = {
    'max_depth' : scope.int(hp.quniform('maxdepth', 4, 100, 1)),  #(round(low, high)/q)*q      
    'learning_rate' : hp.loguniform('learning_rate',-3,0),
    'reg_alpha' : hp.loguniform('reg_alpha',-5,-1),
    'reg_lambda': hp.loguniform('reg_lambda', -6,-1),
    'min_child_weight' : hp.loguniform('min_child_weight',-1,3),
    'objective' : 'reg:linear',
    'seed': 42
}

"""
search space = the ranges in which i want to hyperopt to explore hyper parameters .
"""
best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [19:32:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:52.68636                          
[1]	validation-rmse:52.99561                          
[2]	validation-rmse:53.25743                          
[3]	validation-rmse:53.53700                          
[4]	validation-rmse:53.73219                          
[5]	validation-rmse:53.86720                          
[6]	validation-rmse:53.95353                          
[7]	validation-rmse:53.98075                          
[8]	validation-rmse:54.08940                          
[9]	validation-rmse:54.14860                          
[10]	validation-rmse:54.36566                         
[11]	validation-rmse:54.40962                         
[12]	validation-rmse:54.33925                         
[13]	validation-rmse:54.42301                         
[14]	validation-rmse:54.43636                         
[15]	validation-rmse:54.46243                         
[16]	validation-rmse:54.44100                         
[17]	validation-rmse:54.45470                         
[18]	valid

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [19:32:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:52.90382                                                   
[1]	validation-rmse:53.39039                                                   
[2]	validation-rmse:53.88566                                                   
[3]	validation-rmse:54.24207                                                   
[4]	validation-rmse:54.41688                                                   
[5]	validation-rmse:54.68754                                                   
[6]	validation-rmse:54.77669                                                   
[7]	validation-rmse:54.83546                                                   
[8]	validation-rmse:54.87843                                                   
[9]	validation-rmse:54.86127                                                   
[10]	validation-rmse:54.91400                                                  
[11]	validation-rmse:54.96905                                                  
[12]	validation-rmse:54.94417           

...

[0]	validation-rmse:52.64010                                                    
[1]	validation-rmse:52.59200                                                    
[2]	validation-rmse:52.66851                                                    
[3]	validation-rmse:52.76558                                                    
[4]	validation-rmse:52.92995                                                    
[5]	validation-rmse:53.01150                                                    
[6]	validation-rmse:53.12227                                                    
[7]	validation-rmse:53.17787                                                    
[8]	validation-rmse:53.29032                                                    
[9]	validation-rmse:53.36380                                                    
[10]	validation-rmse:53.41180                                                   
[11]	validation-rmse:53.46588                                                   
[12]	validation-rmse:53.5346

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:03:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:52.16020                                                    
[4]	validation-rmse:52.12348                                                    
[5]	validation-rmse:52.11187                                                    
[6]	validation-rmse:52.10784                                                    
[7]	validation-rmse:52.10902                                                    
[8]	validation-rmse:52.11544                                                    
[9]	validation-rmse:52.13166                                                    
[10]	validation-rmse:52.14160                                                   
[11]	validation-rmse:52.15125                                                   
[12]	validation-rmse:52.16208                                                   
[13]	validation-rmse:52.16793                                                   
[14]	validation-rmse:52.18016                                                   
[15]	validation-rmse:52.2026

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:04:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:52.51717                                                    
[2]	validation-rmse:52.35076                                                    
[3]	validation-rmse:52.27144                                                    
[4]	validation-rmse:52.21819                                                    
[5]	validation-rmse:52.20474                                                    
[6]	validation-rmse:52.21175                                                    
[7]	validation-rmse:52.20815                                                    
[8]	validation-rmse:52.22659                                                    
[9]	validation-rmse:52.24092                                                    
[10]	validation-rmse:52.24816                                                   
[11]	validation-rmse:52.27019                                                   
[12]	validation-rmse:52.29030                                                   
[13]	validation-rmse:52.3183

In [20]:
params_1 = ({
"learning_rate":	0.05084613528431151,
"max_depth":	19,
"min_child_weight":18.04067196472303,
"objective":	"reg:linear",
"reg_alpha":	0.312537524592789,
"reg_lambda":	0.13241132132938369,
"seed":	42})

In [21]:

mlflow.xgboost.autolog()

booster = xgb.train(
params = params_1, #passing parameters in training of a model
dtrain = train, # passing traning data
num_boost_round = 1000, # setting maximum 1000 etirations.
evals = [(valid , 'validation')], #using validation set to control the optimization algorithm from xgboost.
early_stopping_rounds =  50 # if there are 50 or more iteration without improvment on validation error, then the optimization of xgboost will be stop
)

2023/10/15 20:04:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f941e7ff61c94e8a938c05dd59ceb2bb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:04:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:53.06031
[1]	validation-rmse:52.95828
[2]	validation-rmse:52.86884
[3]	validation-rmse:52.79096
[4]	validation-rmse:52.72251
[5]	validation-rmse:52.66428
[6]	validation-rmse:52.61398
[7]	validation-rmse:52.57103
[8]	validation-rmse:52.52959
[9]	validation-rmse:52.49438
[10]	validation-rmse:52.46732
[11]	validation-rmse:52.44247
[12]	validation-rmse:52.42150
[13]	validation-rmse:52.40536
[14]	validation-rmse:52.38900
[15]	validation-rmse:52.37519
[16]	validation-rmse:52.36771
[17]	validation-rmse:52.35684
[18]	validation-rmse:52.35035
[19]	validation-rmse:52.34855
[20]	validation-rmse:52.34479
[21]	validation-rmse:52.34445
[22]	validation-rmse:52.34274
[23]	validation-rmse:52.34055
[24]	validation-rmse:52.34571
[25]	validation-rmse:52.35109
[26]	validation-rmse:52.35096
[27]	validation-rmse:52.35727
[28]	validation-rmse:52.35881
[29]	validation-rmse:52.36541
[30]	validation-rmse:52.36973
[31]	validation-rmse:52.37030
[32]	validation-rmse:52.37177
[33]	validation-rmse

2023/10/15 20:04:36 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/10/15 20:04:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:04:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [25]:
mlflow.xgboost.autolog(disable = True)

In [31]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label = y_train)
    valid = xgb.DMatrix(X_val, label = y_val)

    params_1 = ({
    "learning_rate":	0.05084613528431151,
    "max_depth":	19,
    "min_child_weight":18.04067196472303,
    "objective":	"reg:linear",
    "reg_alpha":	0.312537524592789,
    "reg_lambda":	0.13241132132938369,
    "seed":	42})

    mlflow.log_params(params_1)

    booster = xgb.train(
    params = params_1, #passing parameters in training of a model
    dtrain = train, # passing traning data
    num_boost_round = 1000, # setting maximum 1000 etirations.
    evals = [(valid , 'validation')], #using validation set to control the optimization algorithm from xgboost.
    early_stopping_rounds =  50 # if there are 50 or more iteration without improvment on validation error, then the optimization of xgboost will be stop
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val,y_pred, squared = False)
    mlflow.log_metric('rmse',rmse)

    with open(r'C:\Users\mohdf\Anaconda3\envs\MLOPS\models\preprocessor.b', "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(r'C:\Users\mohdf\Anaconda3\envs\MLOPS\models\preprocessor.b',artifact_path = 'preprocessor')
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")

    

    

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:18:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:53.06031
[1]	validation-rmse:52.95828
[2]	validation-rmse:52.86884
[3]	validation-rmse:52.79096
[4]	validation-rmse:52.72251
[5]	validation-rmse:52.66428
[6]	validation-rmse:52.61398
[7]	validation-rmse:52.57103
[8]	validation-rmse:52.52959
[9]	validation-rmse:52.49438
[10]	validation-rmse:52.46732
[11]	validation-rmse:52.44247
[12]	validation-rmse:52.42150
[13]	validation-rmse:52.40536
[14]	validation-rmse:52.38900
[15]	validation-rmse:52.37519
[16]	validation-rmse:52.36771
[17]	validation-rmse:52.35684
[18]	validation-rmse:52.35035
[19]	validation-rmse:52.34855
[20]	validation-rmse:52.34479
[21]	validation-rmse:52.34445
[22]	validation-rmse:52.34274
[23]	validation-rmse:52.34055
[24]	validation-rmse:52.34571
[25]	validation-rmse:52.35109
[26]	validation-rmse:52.35096
[27]	validation-rmse:52.35727
[28]	validation-rmse:52.35881
[29]	validation-rmse:52.36541
[30]	validation-rmse:52.36973
[31]	validation-rmse:52.37030
[32]	validation-rmse:52.37177
[33]	validation-rmse

c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:19:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
c:\Users\mohdf\Anaconda3\envs\exp-tracking-env\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\mohdf\Anaconda3\envs\exp-trackin